In [1]:
import transformers
import sys
sys.path.append('../')
sys.path.append('./')

from dnabert_datastruct import mask_tokens
from dnabert_datastruct import rnabert_maskwrapper,DNATokenizer
from torch.utils.data import DataLoader
import rna_model
import importlib
import numpy as np
import torch
from transformers import RobertaConfig, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments

In [2]:
tokenizer = DNATokenizer.from_pretrained('dna6')
train_data = rna_model.rna_long_kmer('../data/pre-train/510_6/rna_seq.h5','train',6,tokenizer)
valid_data = rna_model.rna_long_kmer('../data/pre-train/510_6/rna_seq.h5','valid',6,tokenizer)

<class 'dnabert_datastruct.DNATokenizer'>


In [3]:
data_collator = rnabert_maskwrapper(tokenizer,0.15)

In [4]:
test = train_data.__getitem__(1)

In [5]:
output = data_collator([test])

In [9]:
output

{'input_ids': tensor([[   2, 3329, 3318, 2293,   21, 2316, 3150, 1002,   79, 2041, 1798,  108,
          1159, 3115, 2696,  549, 1096,    4,  133, 2685, 2121, 2732,    4,   21,
           218,    5,  158,    4, 2055,  266, 1369, 1562, 2186, 1030,  104,  661,
           343,  429,    4,    4, 2149, 1801, 2709,   39, 1130, 1314, 2444,    4,
           212,  198,  477, 1595,  648,    4,  775,  887, 3034, 2145, 3770, 2454,
          1430, 1450,    4, 2122,  529,  233, 3668, 3372, 3998, 2406, 1308,  773,
          3206, 1617, 3128, 2318, 3933, 1141,  849,  290, 2309, 2329, 2437,  326,
            75,  154,  175, 2991, 2054,  521,  250,    8,    4, 2266,   69,    4,
           118, 1406, 2461,  101,  157, 1557, 1797, 2261, 1513, 1340, 2069, 3850,
          1530, 2501,  620, 1365, 3404, 2519, 2246, 2126, 3576, 2328,    4, 2027,
          2001,  810, 2157,  383, 1609, 1710, 3701,   82, 1805, 2339, 1655, 1017,
          2118,  328,    4, 1642,  548,    4, 1133,  379, 1084, 2389, 3353, 1387,
   

In [8]:
output['input_ids']

tensor([[   2, 3329, 3318, 2293,   21, 2316, 3150, 1002,   79, 2041, 1798,  108,
         1159, 3115, 2696,  549, 1096,    4,  133, 2685, 2121, 2732,    4,   21,
          218,    5,  158,    4, 2055,  266, 1369, 1562, 2186, 1030,  104,  661,
          343,  429,    4,    4, 2149, 1801, 2709,   39, 1130, 1314, 2444,    4,
          212,  198,  477, 1595,  648,    4,  775,  887, 3034, 2145, 3770, 2454,
         1430, 1450,    4, 2122,  529,  233, 3668, 3372, 3998, 2406, 1308,  773,
         3206, 1617, 3128, 2318, 3933, 1141,  849,  290, 2309, 2329, 2437,  326,
           75,  154,  175, 2991, 2054,  521,  250,    8,    4, 2266,   69,    4,
          118, 1406, 2461,  101,  157, 1557, 1797, 2261, 1513, 1340, 2069, 3850,
         1530, 2501,  620, 1365, 3404, 2519, 2246, 2126, 3576, 2328,    4, 2027,
         2001,  810, 2157,  383, 1609, 1710, 3701,   82, 1805, 2339, 1655, 1017,
         2118,  328,    4, 1642,  548,    4, 1133,  379, 1084, 2389, 3353, 1387,
         1016,   70, 1242, 1

In [4]:
len(test)

512

In [2]:
class arg():
    def __init__(self,prb):
        self.mlm_probability = prb

class rnabert_maskwrapper():
    def __init__(self,tokenizer,prob_arg) -> None:
        self.tokenizer = tokenizer
        self.prb = prob_arg
    def __call__(self, batch_entry):
        batch_entry = torch.from_numpy(np.array(batch_entry))
        input,label = mask_tokens(batch_entry,self.tokenizer,arg(self.prb))
        return{'input_ids':input,'labels':label}

In [3]:
importlib.reload(rna_model)
tokenizer = DNATokenizer.from_pretrained('dna6')
train_data = rna_model.rna_kmer('../data/pre-train/510/rna_seq.h5','train',6,tokenizer)
valid_data = rna_model.rna_kmer('../data/pre-train/510/rna_seq.h5','valid',6,tokenizer)
data_collator = rnabert_maskwrapper(tokenizer,0.10)

<class 'dnabert_datastruct.DNATokenizer'>


In [6]:
test_seq = np.array([2,11,12,13,14,15,16,17,18,19,3])
data_collator([test_seq])

tensor([[False, False, False, False, False,  True, False, False, False,  True,
         False]])
tensor([[False, False, False,  True,  True,  True,  True,  True,  True,  True,
         False]])


{'input_ids': tensor([[   2,   11,   12,    4,    4,    4,    4, 1970,    4,    4,    3]]),
 'labels': tensor([[-100, -100, -100, -100, -100,   15, -100, -100, -100,   19, -100]])}